# Phi-4 Mini ONNX Parallel Funksjonskall Veiledning

Denne notatboken viser hvordan du kan bruke Phi-4 Mini med ONNX Runtime GenAI for parallelle funksjonskall. Funksjonskall gjør det mulig for modellen å intelligent bruke eksterne verktøy og API-er basert på brukerforespørsler.

## Oversikt

I denne veiledningen vil du lære:
- Hvordan sette opp Phi-4 Mini med ONNX Runtime GenAI
- Hvordan definere funksjonsskjemaer for å bestille fly og hoteller
- Hvordan bruke veiledet generering med Lark-grammatikk for strukturert output
- Hvordan utføre parallelle funksjonskall for komplekse reisebestillingsscenarier

## Forutsetninger

Før du kjører denne notatboken, sørg for at du har:
- Lastet ned Phi-4 Mini ONNX-modellen
- Installert pakken `onnxruntime-genai`
- Grunnleggende forståelse av konsepter rundt funksjonskall


## Steg 1: Importer nødvendige biblioteker

Først importerer vi de nødvendige bibliotekene for implementeringen av funksjonskall.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Steg 2: Modelloppsett og konfigurasjon

Nå skal vi konfigurere Phi-4 Mini ONNX-modellen. Husk å erstatte banen med din faktiske modellkatalog.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Steg 3: Konfigurer generasjonsparametere

Sett opp generasjonsparametere for å kontrollere modellens utdataoppførsel. Disse innstillingene sikrer deterministiske og fokuserte svar for funksjonskall.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Steg 4: Definer tilgjengelige funksjoner

Her definerer vi funksjonene som vår AI-assistent kan bruke. I dette eksempelet har vi to reiserelaterte funksjoner:
1. **booking_flight_tickets**: For å bestille flybilletter mellom flyplasser
2. **booking_hotels**: For å bestille hotellovernattinger

Funksjonsdefinisjonene følger OpenAI sitt format for funksjonskall.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Steg 5: Hjelpefunksjoner for grammatikkgenerering

Disse hjelpefunksjonene konverterer våre funksjonsdefinisjoner til Lark-grammatikkformat, som brukes av ONNX Runtime GenAI for veiledet generering. Dette sikrer at modellen produserer gyldige funksjonskall i riktig JSON-format.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Steg 6: Test generering av grammatikk

La oss teste funksjonene for generering av grammatikk for å se hvordan de konverterer verktøydefinisjonene våre til riktig format.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Steg 7: Forbered systemprompt og generator

Nå skal vi lage systemprompten som informerer modellen om tilgjengelige verktøy og sette opp generatoren med veiledede generasjonsparametere.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Steg 8: Initialiser Generator med Veiledet Generering

Nå skal vi opprette generatoren med våre konfigurerte parametere og bruke Lark-grammatikken for veiledet generering.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Steg 9: Test parallell funksjonskalling

La oss nå teste oppsettet vårt med et komplekst reisebestillingsscenario som krever kall til flere funksjoner.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generere Funksjonskall

Modellen vil nå generere strukturerte funksjonskall basert på brukerens forespørsel. Takket være veiledet generering, vil output være i gyldig JSON-format som kan kjøres direkte.

**Forventet Output**: Modellen bør generere funksjonskall for:
1. `booking_flight_tickets` med detaljer fra Beijing (PEK) til Paris (CDG)
2. `booking_hotels` med detaljer for overnatting i Paris

Kjør cellen nedenfor for å se den direkte genereringen:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Konklusjon

🎉 **Gratulerer!** Du har med suksess implementert parallelle funksjonskall med Phi-4 Mini ved hjelp av ONNX Runtime GenAI.

### Hva du har lært:

1. **Modelloppsett**: Hvordan konfigurere Phi-4 Mini med ONNX Runtime GenAI
2. **Funksjonsdefinisjon**: Hvordan definere funksjonsskjemaer for AI-funksjonskall
3. **Veiledet generering**: Hvordan bruke Lark-grammatikk for strukturert output-generering
4. **Parallelle funksjonskall**: Hvordan håndtere komplekse forespørsler som krever flere funksjonskall

### Viktige fordeler:

- ✅ **Strukturert output**: Veiledet generering sikrer gyldige JSON-funksjonskall
- ✅ **Parallell behandling**: Håndter flere funksjonskall i én forespørsel
- ✅ **Høy ytelse**: ONNX Runtime gir optimalisert inferens
- ✅ **Fleksibelt skjema**: Enkelt å legge til eller endre funksjonsdefinisjoner

### Ressurser:

- [Phi-4 Mini Dokumentasjon](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentasjon](https://onnxruntime.ai/docs/genai/)
- [Beste praksis for funksjonskall](https://platform.openai.com/docs/guides/function-calling)



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
